In [1]:
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
def fill_nan(dataframe):
    dataframe['precipitation'] = dataframe['precipitation'].fillna(0)
    dataframe = dataframe.fillna(method='bfill')
    return dataframe


def seperate_datetime(dataframe):
    week_list = []
    year = []
    month = []
    for date in dataframe.date:
        year_point, month_point, day_point = date.split('-')
        year.append(int(year_point) - 2017)
        month.append(int(month_point))
    dataframe['year'] = year
    dataframe['month'] = month
    for day in dataframe['date']:
        num = pd.date_range(day, day, freq='D').to_series()
        week_list.append(int(num.dt.dayofweek[0]))
    dataframe['day'] = week_list
    dataframe = dataframe.drop(['date'], axis=1)
    return dataframe


def weekday_onehotcode(dataframe):
    mon = dataframe[dataframe['day'] == 0]
    mon['day'] = 'mon'
    tue = dataframe[dataframe['day'] == 1]
    tue['day'] = 'tue'
    wed = dataframe[dataframe['day'] == 2]
    wed['day'] = 'wed'
    thu = dataframe[dataframe['day'] == 3]
    thu['day'] = 'thu'
    fri = dataframe[dataframe['day'] == 4]
    fri['day'] = 'fri'
    sat = dataframe[dataframe['day'] == 5]
    sat['day'] = 'sat'
    sun = dataframe[dataframe['day'] == 6]
    sun['day'] = 'sun'
    dataframe = pd.concat([mon, tue, wed, thu, fri, sat, sun], axis=0)
    dataframe = pd.get_dummies(dataframe)
    return dataframe


def month_onehotcode(dataframe):
    a = dataframe[dataframe['month'] == 1]
    a['month'] = 'a'
    b = dataframe[dataframe['month'] == 2]
    b['month'] = 'b'
    c = dataframe[dataframe['month'] == 3]
    c['month'] = 'c'
    d = dataframe[dataframe['month'] == 4]
    d['month'] = 'd'
    e = dataframe[dataframe['month'] == 5]
    e['month'] = 'e'
    f = dataframe[dataframe['month'] == 6]
    f['month'] = 'f'
    g = dataframe[dataframe['month'] == 7]
    g['month'] = 'g'
    h = dataframe[dataframe['month'] == 8]
    h['month'] = 'h'
    i = dataframe[dataframe['month'] == 9]
    i['month'] = 'i'
    j = dataframe[dataframe['month'] == 10]
    j['month'] = 'j'
    k = dataframe[dataframe['month'] == 11]
    k['month'] = 'k'
    l = dataframe[dataframe['month'] == 12]
    l['month'] = 'l'
    dataframe = pd.concat([a, b, c, d, e, f, g, h, i, j, k, l], axis=0)
    dataframe = pd.get_dummies(dataframe)
    return dataframe


def year_onehotcode(dataframe):
    a = dataframe[dataframe['year'] == 1]
    a['year'] = 'a'
    b = dataframe[dataframe['year'] == 2]
    b['year'] = 'b'
    c = dataframe[dataframe['year'] == 3]
    c['year'] = 'c'
    dataframe = pd.concat([a, b, c], axis=0)
    dataframe = pd.get_dummies(dataframe)
    return dataframe


def rental_rate(dataframe):
    y1 = dataframe[dataframe['year'] == 1]['rental'] * 2.3
    y2 = dataframe[dataframe['year'] == 2]['rental'] * 1.2
    y3 = dataframe[dataframe['year'] == 3]['rental']
    new = pd.concat([y1, y2, y3], axis=0).to_frame()
    dataframe['rental'] = new['rental']
    return True


def NMAE(true, pred):
    score = np.mean(np.abs(true - pred) / true)
    return score


def job(x,y,size):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=size, random_state=156)
    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.1, max_depth = 5, n_estimators = 500)
    xg_reg.fit(X_train, y_train)
    pred = xg_reg.predict(X_test)
    pred = np.ceil(pred)
    score = NMAE(y_test, pred)
    print(len(X_train),len(y_train))
    print(score)
    # print('점수: ', xg_reg.score(X_test, y_test))
    return score

In [3]:
bicycle = pd.read_csv('train.csv')
bicycle = seperate_datetime(bicycle)
bicycle = fill_nan(bicycle)
rental_rate(bicycle)
bicycle = weekday_onehotcode(bicycle)
# bicycle = year_onehotcode(bicycle)
# bicycle = month_onehotcode(bicycle)
bicycle.sort_index()
y = bicycle['rental']
bicycle = bicycle.drop(['rental'], axis=1)
bicycle

,precipitation,temp_mean,temp_highest,temp_lowest,PM10,PM2.5,humidity,sunshine_sum,sunshine_rate,wind_mean,wind_max,year,month,day_fri,day_mon,day_sat,day_sun,day_thu,day_tue,day_wed
0,0.0,-1.3,3.8,-5.1,34.0,17.0,39.1,8.3,86.5,1.4,3.8,1,1,0,1,0,0,0,0,0
7,0.9,1.3,4.0,-1.2,49.0,37.0,51.9,0.5,5.2,1.8,7.1,1,1,0,1,0,0,0,0,0
14,0.2,4.7,7.7,1.8,63.0,45.0,63.9,6.5,65.7,1.4,3.3,1,1,0,1,0,0,0,0,0
21,3.3,0.2,4.0,-5.3,38.0,21.0,56.5,0.7,7.0,1.5,5.1,1,1,0,1,0,0,0,0,0
28,0.0,-8.5,-4.7,-11.6,39.0,16.0,35.0,8.9,87.3,2.7,6.2,1,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,0.0,-0.6,2.6,-3.6,26.0,16.0,59.6,1.5,15.3,2.2,4.5,3,11,0,0,0,1,0,0,0
1069,0.0,4.5,9.1,1.2,45.0,33.0,66.9,6.0,61.9,1.8,4.3,3,12,0,0,0,1,0,0,0
1076,2.9,-1.1,2.0,-6.0,36.0,25.0,74.9,1.1,11.5,2.9,6.1,3,12,0,0,0,1,0,0,0
1083,0.0,-3.7,1.1,-8.2,27.0,16.0,50.4,9.1,94.8,2.0,4.4,3,12,0,0,0,1,0,0,0


In [32]:
T = bicycle['temp_mean']  # 섭씨
V = bicycle['wind_mean']  # 바람
R = bicycle['humidity']  # 습도
H = (bicycle['temp_mean'] * 1.8) + 32  # 화씨
feel_degree = 13.12 + 0.6215 * T - 11.37 * V**0.16 + 0.3965 * V**0.16
badfeel_degree = 9/5 * T - 0.55 * (1 - R) * (9/5 * T - 26) + 32
heat_degree = -42.379 + 2.05901523 * H + 10.14333127 * R - 0.22475541 * H * R - 6.83783 * 10**-3 * H**2 - 5.481717 * 10**-2 * R**2 + 1.22874 * 10**-3 * H**2 * R + 8.5282 * 10**-4 * H * R**2 - 1.99 * 10**-6 * H**2 * R**2
bicycle['cold_feel'] = bicycle['temp_lowest']*bicycle['wind_max']
bicycle['feel_degree'] = feel_degree
# bicycle['badfeel_degree'] = badfeel_degree
# bicycle['heat_degree'] = heat_degree

In [33]:
col_list = list(bicycle.columns)
for i in range(len(col_list)):
    for j in range(i, len(col_list)):
        bicycle[f'{col_list[i]}*{col_list[j]}'] = bicycle[col_list[i]] * bicycle[col_list[j]]

x = bicycle

size = 0
score = 0
for i in range(8):
    size += 0.1
    score += job(x,y,size)
print('평균 : ', score/8)

985 985
0.13915552862904013
876 876
0.14637463470795845
766 766
0.1603768929048969
657 657
0.16660517237194383
547 547
0.19504293694329783
438 438
0.21611952201291007
328 328
0.2162038839841885
219 219
0.230168723316352
평균 :  0.18375591185882348


In [34]:
bicycle_test = pd.read_csv('test.csv')
bicycle_test_date = bicycle_test['date']
bicycle_test = seperate_datetime(bicycle_test)
bicycle_test = fill_nan(bicycle_test)  # 바로 뒤에 값으로 결측치를 채우므로 seperate 밑이 와야함
bicycle_test = weekday_onehotcode(bicycle_test)
# bicycle_test = month_onehotcode(bicycle_test)
bicycle_test = bicycle_test.sort_index()
bicycle_test

,precipitation,temp_mean,temp_highest,temp_lowest,PM10,PM2.5,humidity,sunshine_sum,sunshine_rate,wind_mean,wind_max,year,month,day_fri,day_mon,day_sat,day_sun,day_thu,day_tue,day_wed
0,0.0,-4.2,1.6,-9.8,30.0,17.0,64.0,6.5,67.7,2.0,4.1,4,1,1,0,0,0,0,0,0
1,0.0,-5.0,-1.4,-8.4,34.0,12.0,38.5,9.0,93.8,2.6,5.4,4,1,0,0,1,0,0,0,0
2,0.0,-5.6,-2.0,-9.1,39.0,14.0,45.0,5.5,56.7,2.0,4.5,4,1,0,0,0,1,0,0,0
3,0.0,-3.5,0.3,-8.4,40.0,23.0,51.4,4.6,47.4,1.7,3.2,4,1,0,1,0,0,0,0,0
4,0.0,-5.5,-2.1,-9.9,30.0,17.0,52.8,8.6,88.7,2.9,5.7,4,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,0.0,-7.6,-3.9,-12.9,33.0,20.0,60.9,3.8,39.6,1.7,3.1,4,12,0,1,0,0,0,0,0
361,0.0,-4.1,-0.9,-8.5,51.0,38.0,73.8,1.7,17.7,2.2,3.1,4,12,0,0,0,0,0,1,0
362,0.2,0.4,5.9,-3.8,66.0,49.0,72.9,1.8,18.8,2.6,5.9,4,12,0,0,0,0,0,0,1
363,0.0,-3.9,0.2,-6.8,30.0,17.0,48.5,7.3,76.0,3.3,6.6,4,12,0,0,0,0,1,0,0


In [35]:
T = bicycle_test['temp_mean']  # 섭씨
V = bicycle_test['wind_mean']  # 바람
R = bicycle_test['humidity']  # 습도
H = (bicycle_test['temp_mean'] * 1.8) + 32  # 화씨
feel_degree = 13.12 + 0.6215 * T - 11.37 * V**0.16 + 0.3965 * V**0.16
badfeel_degree = 9/5 * T - 0.55 * (1 - R) * (9/5 * T - 26) + 32
heat_degree = -42.379 + 2.05901523 * H + 10.14333127 * R - 0.22475541 * H * R - 6.83783 * 10**-3 * H**2 - 5.481717 * 10**-2 * R**2 + 1.22874 * 10**-3 * H**2 * R + 8.5282 * 10**-4 * H * R**2 - 1.99 * 10**-6 * H**2 * R**2
bicycle_test['cold_feel'] = bicycle_test['temp_lowest']*bicycle_test['wind_max']
bicycle_test['feel_degree'] = feel_degree
# bicycle_test['badfeel_degree'] = badfeel_degree
# bicycle_test['heat_degree'] = heat_degree

In [36]:
col_list = list(bicycle_test.columns)
for i in range(len(col_list)):
    for j in range(i, len(col_list)):
        bicycle_test[f'{col_list[i]}*{col_list[j]}'] = bicycle_test[col_list[i]] * bicycle_test[col_list[j]]

x2 = bicycle_test

In [37]:
X_train = x
y_train = y
X_test = x2
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.1, max_depth = 5, n_estimators = 500)
xg_reg.fit(X_train, y_train)
pred = xg_reg.predict(X_test) * 1.2
pred = pd.DataFrame(pred, columns=['rental'])
print(pred)
pred = np.ceil(pred)
result = pd.concat([bicycle_test_date, pred],axis=1)
result.to_csv('sample_submissoin.csv',index=False)
print(result)

           rental
0    27249.884766
1    24066.632812
2    20431.974609
3    28904.380859
4    25976.111328
..            ...
360  34054.476562
361  26446.404297
362  47553.789062
363  38981.242188
364  37680.292969

[365 rows x 1 columns]
           date   rental
0    2021-01-01  27250.0
1    2021-01-02  24067.0
2    2021-01-03  20432.0
3    2021-01-04  28905.0
4    2021-01-05  25977.0
..          ...      ...
360  2021-12-27  34055.0
361  2021-12-28  26447.0
362  2021-12-29  47554.0
363  2021-12-30  38982.0
364  2021-12-31  37681.0

[365 rows x 2 columns]


In [38]:
col_list = list(bicycle_test.columns)
for i in range(len(col_list)):
    for j in range(i, len(col_list)):
        bicycle_test[f'{col_list[i]}*{col_list[j]}'] = bicycle_test[col_list[i]] * bicycle_test[col_list[j]]

x2 = bicycle_test